In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [3]:
import sys; sys.path.insert(0, '../..') # add parent folder path where lib folder is

In [4]:
import ray
import pandas as pd
from utils import helper, config, rayer
from ml.models.ensemble import Ensemble
from ml.xai.model.explainable import Explainable
from sklearn.preprocessing import StandardScaler


from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
from ml.models import common


In [5]:
rayer.get_global_cluster(num_cpus=5)
!ray status --address='raycluster-autoscaler-head-svc.dev.svc.cluster.local:6379'

======== Autoscaler status: 2022-11-17 05:59:40.112424 ========
Node status
---------------------------------------------------------------
Healthy:
 1 head-group
 2 small-group
Pending:
 (no pending nodes)
Recent failures:
 (no failures)

Resources
---------------------------------------------------------------
Usage:
 0.0/13.0 CPU
 0.00/46.566 GiB memory
 0.00/13.742 GiB object_store_memory

Demands:
 {'CPU': 1}: 5+ from request_resources()


In [3]:
!ray status --address='raycluster-autoscaler-head-svc.dev.svc.cluster.local:6379'

======== Autoscaler status: 2022-11-17 08:37:13.186794 ========
Node status
---------------------------------------------------------------
Healthy:
 1 head-group
 3 small-group
Pending:
 (no pending nodes)
Recent failures:
 (no failures)

Resources
---------------------------------------------------------------
Usage:
 2.0/7.0 CPU
 0.00/24.214 GiB memory
 0.00/7.114 GiB object_store_memory

Demands:
 {'CPU': 1}: 5+ from request_resources()


In [7]:
# ray.autoscaler.sdk.request_resources(num_cpus=0)

In [8]:
df_X, df_y = helper.get_covid_dataset()
df_X = df_X.drop(['location'], axis = 1)
# X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.33)

In [9]:
ss = StandardScaler()
df_X_scalar = pd.DataFrame(ss.fit_transform(df_X), columns=df_X.columns)


In [15]:
list_base_models = ['BriskXGBoost', 'SlugXGBoost', 'SlugANN', 'SlugRF', 'SlugKNN', 'BriskBagging']

ensemble_set = Ensemble(df_X=df_X_scalar,
                        df_y=df_y,
                        #list_base_models = list_base_models,
                        n_trials=1000, ### for all models
                        boosted_round=100, ### for tree models only
                        epochs=150, ### for DNN models

                        ensemble_boosted_round=10, ### for ensemble model which is also a free model
                        ensemble_n_trials=3,
                                )

In [16]:
threshold=None
ensemble_set.fetch_models(threshold=threshold)
ensemble_set.fit()

2022-11-17T06:04:21CST : INFO : ensemble : fetch_models : 115 : Message : Ensemble: starting discovery process for models [<ml.models.base.brisk_xgboost.BriskXGBoost object at 0x7fa72f3931f0>, <ml.models.base.slug_xgboost.SlugXGBoost object at 0x7fa72f393310>, <ml.models.base.slug_ann.SlugANN object at 0x7fa7e37f62e0>, <ml.models.base.slug_rf.SlugRF object at 0x7fa7e37f6520>, <ml.models.base.slug_knn.SlugKNN object at 0x7fa7e37f6df0>, <ml.models.base.brisk_bagging.BriskBagging object at 0x7fa7e37da0d0>]
(__run_discoveries__ pid=167, ip=10.10.121.96) 2022-11-17T04:04:21PST : INFO : slug_rf : __discover_model__ : 101 : Message : slug_rf: Starting training for trials:1000, max depth: 30, max estimators 1500
(__run_discoveries__ pid=166, ip=10.10.121.96) 2022-11-17T04:04:21PST : INFO : slug_ann : __discover_model__ : 225 : Message : slug_ann: Starting train for trials:1000 with epochs: 150
(__run_discoveries__ pid=29958) 2022-11-17T04:04:21PST : INFO : brisk_xgboost : __discover_model__ : 

(scheduler +4m46s) Adding 1 node(s) of type small-group.
(scheduler +4m52s) Resized to 15 CPUs.


(__run_discoveries__ pid=162, ip=10.10.127.121) 2022-11-17T04:04:41PST : INFO : slug_knn : __discover_model__ : 111 : Message : slug_knn: Number of trials: 1000
(__run_discoveries__ pid=162, ip=10.10.127.121) 2022-11-17T04:04:41PST : INFO : slug_knn : __discover_model__ : 113 : Message : Best trial:6
(__run_discoveries__ pid=162, ip=10.10.127.121) 2022-11-17T04:04:41PST : INFO : slug_knn : __discover_model__ : 115 : Message :   Params: 
(__run_discoveries__ pid=162, ip=10.10.127.121) 2022-11-17T04:04:41PST : INFO : slug_knn : __discover_model__ : 117 : Message :     n_neighbors 10
(__run_discoveries__ pid=162, ip=10.10.127.121) 2022-11-17T04:04:41PST : INFO : slug_knn : __discover_model__ : 117 : Message :     weights uniform
(__run_discoveries__ pid=162, ip=10.10.127.121) 2022-11-17T04:04:41PST : INFO : slug_knn : __discover_model__ : 117 : Message :     algorithm kd_tree
(__run_discoveries__ pid=162, ip=10.10.127.121) 2022-11-17T04:04:41PST : INFO : slug_knn : __discover_model__ : 13

ConnectionError: GRPC connection failed: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.FAILED_PRECONDITION
	details = "'NoneType' object is not iterable"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:172.20.14.107:10001 {grpc_message:"\'NoneType\' object is not iterable", grpc_status:9, created_time:"2022-11-17T08:00:57.730024572-06:00"}"
>

In [ ]:
ensemble_set.base_model_scores

Unexpected exception:
Traceback (most recent call last):
  File "/home/conda/admin/807c952d1ebb2e498414dc4b80a90b96c80ef2ef433f07c7a7381b5e38e2b5fa-20221110-224801-275852-64-wasif-dev-ray/lib/python3.9/site-packages/ray/util/client/logsclient.py", line 67, in _log_main
    for record in log_stream:
  File "/home/conda/admin/807c952d1ebb2e498414dc4b80a90b96c80ef2ef433f07c7a7381b5e38e2b5fa-20221110-224801-275852-64-wasif-dev-ray/lib/python3.9/site-packages/grpc/_channel.py", line 426, in __next__
    return self._next()
  File "/home/conda/admin/807c952d1ebb2e498414dc4b80a90b96c80ef2ef433f07c7a7381b5e38e2b5fa-20221110-224801-275852-64-wasif-dev-ray/lib/python3.9/site-packages/grpc/_channel.py", line 826, in _next
    raise self
grpc._channel._MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.NOT_FOUND
	details = "Logstream proxy failed to connect. Channel for client a2bfb988412d4b16901f762ec1a431c1 not found."
	debug_error_string = "UNKN

In [28]:
attr_algos = ['IG', 'SHAP', 'GradientSHAP']

In [29]:
base_knn = ensemble_set.base_model_scores[0][2].best_fit
base_knn

In [30]:
f = lambda x: base_knn.predict(df_X)[:,1]
f

<function __main__.<lambda>(x)>

In [29]:
med = df_X.median().values.reshape((1, df_X.shape[1]))
med

array([[6.46857170e+04, 8.71549400e+06, 7.40800000e+01, 8.52561100e+06,
        4.57470000e+04, 8.10000000e+01, 2.01900000e+03, 7.84305400e-04,
        1.03176250e-01, 1.51096200e-03, 0.00000000e+00, 2.39895150e-03,
        3.61773320e-03, 3.87874450e-03, 4.34872700e-04, 4.11234510e-03,
        1.30209875e-02, 8.07507400e-04, 3.18185793e-02, 8.98164190e-03,
        1.02990850e-03, 0.00000000e+00, 3.58518310e-03, 2.43388300e-04,
        1.79446230e-03, 7.99664400e-03, 0.00000000e+00, 2.06014499e-02,
        4.67140800e-04, 1.71194770e-03, 2.15557638e-01, 1.92910548e-02,
        2.81162137e-02, 1.78403544e-02, 3.39738527e-02, 1.67354600e-04,
        1.42763181e-02, 4.02276040e-03, 6.98318887e-01, 2.08346400e+01]])

In [32]:
import shap
#f = lambda x: knn.predict_proba(x)[:,1]
#med = X_train.median().values.reshape((1,X_train.shape[1]))
explainer = shap.KernelExplainer(base_knn.predict, med)
#shap_values_single = explainer.shap_values(df_X)
#shap.force_plot(explainer.expected_value, shap_values_single, df_X.iloc[0,:])

Provided model function fails when applied to the provided data set.


TypeError: ('Expecting data to be a DMatrix object, got: ', <class 'numpy.ndarray'>)

In [17]:
#pd.DataFrame(shap_values_single)

df_shapley_sores = pd.DataFrame(shap_values_single, columns=df_X.columns)
df_shapley_sores_list = df_shapley_sores.abs().mean().values #sort_values(ascending=False).values
df_shapley_sores_list

array([1.28138464e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 4.99681526e-01, 0.00000000e+00, 0.00000000e+00,
       1.35383376e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 6.86163052e-04, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.39872550e-01,
       0.00000000e+00, 0.00000000e+00, 7.47050275e+00, 2.44781690e+00,
       0.00000000e+00, 0.00000000e+00, 7.81468026e+00, 0.00000000e+00,
       5.09664526e-01, 0.00000000e+00, 2.44484819e+01, 0.00000000e+00])

(scheduler +1m35s) Resized to 13 CPUs.


In [49]:
shap.force_plot(explainer.expected_value, shap_values_single, df_X)

In [41]:
shap_values_single

array([[ 0.        ,  0.11622893,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        , -0.12380398, -0.00908862, ...,  0.        ,
         0.        , -0.0114041 ],
       [ 0.        ,  0.13067563,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.12522929,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [18]:
models = [model.best_fit for model in ensemble_set.base_models]
models

 Sequential(
   (0): Linear(in_features=40, out_features=229, bias=True)
   (1): ReLU()
   (2): Dropout(p=0.20239089138733532, inplace=False)
   (3): Linear(in_features=229, out_features=1, bias=True)
   (4): ReLU()
 ),
 RandomForestRegressor(max_depth=14, min_samples_leaf=22, min_samples_split=137,
                       n_estimators=720, random_state=0),
 KNeighborsRegressor(algorithm='brute', n_neighbors=10),
 BaggingRegressor(n_estimators=30)]

In [16]:
models[4:5]
# models = models[4]
# models

[KNeighborsRegressor(n_neighbors=9)]

In [22]:
models[5]

BaggingRegressor(n_estimators=44)

In [12]:
ex = Explainable(ensemble_set)

In [1]:
attr_algos = ['IG', 'SHAP', 'GradientSHAP']
#ex.get_attr(attr_algos)

In [14]:
ex.df_scores

,brisk_xgb_shap,brisk_xgb_shap,brisk_xgb_shap,slug_xgb_shap,slug_xgb_shap,slug_xgb_shap,slug_ann_IG,slug_ann_SHAP,slug_ann_GradientSHAP,slug_rf_shap,slug_rf_shap,slug_rf_shap,slug_knn_shap,slug_knn_shap,slug_knn_shap,brisk_bagging_shap,brisk_bagging_shap,brisk_bagging_shap
new_cases_per_million,0.000685,0.000685,0.000685,0.000000,0.000000,0.000000,35.760640,32.511722,52.321301,0.0,0.0,0.0,99.761187,99.524513,99.293513,544.348296,544.409780,544.539735
population_cov,0.003111,0.003111,0.003111,0.000918,0.000918,0.000918,6.930936,5.187041,6.850191,0.0,0.0,0.0,5.782943,6.224602,6.064686,6.744558,6.846390,6.846132
life_expectancy_cov,0.000018,0.000018,0.000018,0.000000,0.000000,0.000000,31.908143,27.943693,43.172096,0.0,0.0,0.0,40.796758,41.315272,40.601357,9.699675,9.935441,9.576456
Population,0.000000,0.000000,0.000000,0.000306,0.000306,0.000306,6.942685,5.204729,6.953215,0.0,0.0,0.0,6.380263,6.544700,6.028896,19.322730,19.477435,19.492469
Area_km2,0.001536,0.001536,0.001536,0.000305,0.000305,0.000305,1.980677,1.754682,2.293945,0.0,0.0,0.0,8.512688,8.258834,7.984152,15.493113,15.523685,15.612539
Density_km2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.862189,8.423567,24.821980,0.0,0.0,0.0,5.699390,4.902428,4.607391,20.793688,20.881263,20.631375
Year_x,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.073414,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Meningitis,0.000020,0.000020,0.000020,0.000000,0.000000,0.000000,29.624915,20.634687,23.724682,0.0,0.0,0.0,18.771857,19.113895,18.698931,4.299620,4.134574,4.240839
Neoplasms,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,28.553472,26.482165,43.047684,0.0,0.0,0.0,56.198154,56.096681,56.258803,20.065283,20.350275,20.220420
"Fire, heat, and hot substances",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.979219,6.951856,11.611641,0.0,0.0,0.0,16.382295,16.508486,17.307379,9.511534,9.485713,9.628903


In [14]:
trained_models = []

for base_model in ensemble_set.base_models:
    if base_model.best_fit:
        trained_models.append(base_model.best_fit)


In [36]:
df_X_id = ray.put(df_X)

In [37]:
df_X_temp = ray.get(df_X_id)

In [40]:
import shap
explainer = shap.Explainer(trained_models[0])
shap_values = explainer(df_X_temp)


TypeError: 'NoneType' object is not callable

(scheduler +32m52s) Resized to 19 CPUs.
(scheduler +32m52s) Removing 2 nodes of type small-group (idle).
(scheduler +33m6s) Resized to 15 CPUs.
(scheduler +33m54s) Removing 5 nodes of type small-group (idle).
(scheduler +34m5s) Resized to 5 CPUs.


In [26]:
models

Sequential(
  (0): Linear(in_features=40, out_features=64, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.2996592516681684, inplace=False)
  (3): Linear(in_features=64, out_features=83, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.2996592516681684, inplace=False)
  (6): Linear(in_features=83, out_features=38, bias=True)
  (7): ReLU()
  (8): Dropout(p=0.2996592516681684, inplace=False)
  (9): Linear(in_features=38, out_features=129, bias=True)
  (10): ReLU()
  (11): Dropout(p=0.2996592516681684, inplace=False)
  (12): Linear(in_features=129, out_features=48, bias=True)
  (13): ReLU()
  (14): Dropout(p=0.2996592516681684, inplace=False)
  (15): Linear(in_features=48, out_features=34, bias=True)
  (16): ReLU()
  (17): Dropout(p=0.2996592516681684, inplace=False)
  (18): Linear(in_features=34, out_features=1, bias=True)
  (19): ReLU()
)

In [27]:
models = trained_models[0]

In [ ]:
ex = Explainable(df_X, models)

In [ ]:
attr_algos = ['IG']
ex.get_attr(attr_algos)